# Assignment 1 
### Team Memebers :
 - Bhargav Patel - CS23MTECH11026
 - Manan Patel - CS23MTECH14006
 - Hrishikesh Hemke - CS23MTECH14003
 - Yug Patel - CS23MTECH14019

In [1]:
import numpy as np
from sympy import Matrix
import pandas as pd

In [2]:
class Simplex_Algo():

    filePath = None
    A =None
    b =None
    z =None
    c =None
    nArr =None

    def __init__(self,filePath):
        nArr = pd.read_csv(filePath, header=None)

        # Extracting the data
        self.z = nArr.iloc[0, :-1].values.reshape(-1, 1)
        self.c = nArr.iloc[1, :-1].values
        self.b = nArr.iloc[2:, -1:].values
        self.A = nArr.iloc[2:, :-1].values
        self.nArr =nArr.to_numpy()


    # Step 2: Function to Move towards a vertex from a given point z
    def Finding_initial_vertex(self,z):

        A1 = self.A[np.where((np.dot(self.A, z)) == self.b)[0]]
        B1 = self.b[np.where((np.dot(self.A, z)) == self.b)[0]]

        print('Initial vertex Tour:')

        while A1.shape[0] != self.A.shape[1]:

            epsilon = 1e-1

            cost = np.dot(self.c, z)
            
            flat_eles = list(map(float, z.flatten()))
            print(f'z: {tuple((flat_eles))}; cost: {np.round(cost[0], decimals=2)}')

           
            A1 = self.A[np.where((np.dot(self.A, z)) == self.b)[0]]
            B1 = self.b[np.where((np.dot(self.A, z)) == self.b)[0]]

            n = A1.shape[0]
            
            while A1.shape[0] <= n:
            
                S = None
                if A1.shape[0] == 0:
                    S = np.array([[1]]+[[0]]*(len(z)-1))
                
                else:
                    null_sp = np.array(Matrix(A1).nullspace(), dtype=float)
                    for null_ve in null_sp:
                        temp = np.dot(A1, np.round(z + epsilon * null_ve, decimals=2)) == B1
                        if np.all(temp):
                            S = null_ve
                            break
                        
                if S is None:
                    print("No space found")
                    epsilon = epsilon*1e-1
                    continue

                z_temp = z + epsilon*S

                if all(np.dot(self.A, np.round(z_temp,decimals=2)) <= self.b):
                    z = z_temp
                else:
                    print("Invalid")
                    epsilon = epsilon*1e-1
                    continue
                cost = np.dot(self.c, z) 
                
                A1 = self.A[np.where((np.dot(self.A, np.round(z, decimals = 2))) == self.b)[0]]
                B1 = self.b[np.where((np.dot(self.A, np.round(z, decimals = 2))) == self.b)[0]]


        z = np.round(z, decimals = 2)
        
        cost = np.dot(self.c, z)
        flat_eles = list(map(float, z.flatten()))
        print(f'z: {tuple((flat_eles))}; cost: {np.round(cost[0], decimals=2)}')
        return z


    # Step 3: Function to move towards an optimal vertex
    def vertex_marching(self,z):

        A1 = self.A[np.where((np.dot(self.A, z)) == self.b)[0]]

        epsilons = np.linalg.inv(A1.T).dot(self.c)

        print("\nJourney to an optimal vertex:")

        while False in (epsilons >= 0):

            beta = 1e-1

            cost = np.dot(self.c, z)
            flat_eles = list(map(float, z.flatten()))
            print(f'z: {tuple(flat_eles)}; cost: {np.round(cost[0], decimals=2)}')

            A1inv = np.linalg.inv(A1)
            C = None
            for i in range(len(epsilons)):
                if epsilons[i] < 0:
                    C = A1inv[:, i:i+1]
                    break
            
            if C is None:
                break
            ztemp = z

            A1 = self.A[np.where((np.dot(self.A, np.round(ztemp, decimals=2))) == self.b)[0]]


            while A1.shape[0] != self.A.shape[1] or not np.any(np.round(ztemp, decimals=2) != z):
                ztemp = ztemp + beta * (-C)

                if not (all(np.dot(self.A, np.round(ztemp,decimals=2)) <= self.b)):
                    beta *= 1e-1
                    continue

                A1 = self.A[np.where((np.dot(self.A, np.round(ztemp, decimals=2))) == self.b)[0]]

                cost = np.dot(self.c, ztemp)

            z = np.round(ztemp, decimals=2)
            epsilons = np.linalg.inv(A1.T).dot(self.c)

        
        z = np.round(z, decimals = 2)
        cost = np.dot(self.c, z)
        flat_eles = list(map(float, z.flatten()))
        print(f'z: {tuple((flat_eles))}; cost: {np.round(cost[0], decimals=2)}')

        return z

    def fit(self):
        """
        Implements the initial steps of the Simplex algorithm.

        :param file_path: Path to the input CSV file.
        :return: Sequence of vertices visited and the value of the objective function at each vertex.
        """

        point = tuple((map(float, list(self.z.ravel()))))

        # Step 1: Check if the initial point is feasible
        if not (all(np.dot(self.A, self.z) <= self.b)):
            print(f"The initial point z: {point} is not feasible.\n")
            return None
        else:
            print(f"The initial point z: {point} is feasible\n")

        # Step 2: Finding an initial vertex
        vertex = self.Finding_initial_vertex(self.z)
        
        # Step 3: Initial vertex to an optimal vertex
        optimal_vertex = self.vertex_marching(vertex)
        
        return optimal_vertex



In [5]:
algo = Simplex_Algo('Assignment1.csv')
optimal_solution = algo.fit()
if optimal_solution is not None:
    optimal_solution = Matrix(optimal_solution)
    print("\nOptimal cost of the objective function : ")
optimal_solution

The initial point z: (5.0, 0.0, 0.0) is feasible

Initial vertex Tour:
z: (5.0, 0.0, 0.0); cost: 10

Journey to an optimal vertex:
z: (5.0, 0.0, 0.0); cost: 10
z: (5.0, 0.0, 2.5); cost: 15.0

Optimal cost of the objective function : 


Matrix([
[5.0],
[  0],
[2.5]])

In [6]:
# Test input for above Output

# 5,0,0,0
# 2,-1,2,0
# 2,1,0,10
# 1,2,-2,20
# 0,1,2,5
# -1,0,0,0
# 0,-1,0,0
# 0,0,-1,0